In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, cv2
import random
import seaborn as sns
import numba

In [2]:
def displacement():
    x0, y0, z0 = np.mean(x[int(count/2.):count,:],axis=0), np.mean(y[int(count/2.):count,:],axis=0), np.mean(z[int(count/2.):count,:],axis=0)
    num_0 = np.argmin(x0**2 + y0**2 + z0**2)
    start = int(count/2.)
    x0, y0, z0 = np.mean(x[int(count/2.):count,num_0]), np.mean(y[int(count/2.):count,num_0]), np.mean(z[int(count/2.):count,num_0])

    x_, y_, z_ = np.roll(x[:,num_0], 1), np.roll(y[:,num_0], 1), np.roll(z[:,num_0], 1)
    sx, sy, sz = x[start:,num_0] - x_[start:], y[start:,num_0] - y_[start:], z[start:,num_0] - z_[start:]
    s = np.sqrt(sx**2 + sy**2 + sz**2)

#     plt.figure(1)
#     plt.plot(np.arange(start,s.shape[0]+start), s)
#     plt.plot(np.arange(start,s.shape[0]+start), sx, label='x')
#     plt.plot(np.arange(start,s.shape[0]+start), sy, label='y')
#     plt.plot(np.arange(start,s.shape[0]+start), sz, label='z')
#     plt.ylabel('s=|$r_2 - r_1$|')
#     plt.xlabel('t')
#     plt.legend()
#     plt.grid()
#     plt.savefig('s.png')


    t = 50
    st, stx, sty, stz = np.array([]), np.array([]), np.array([]), np.array([])
    xt = np.array([])
    for i in range(1,count-t-start):
        st, stx, sty, stz = np.append(st, np.mean(s[i:i+t])), np.append(stx, np.mean(sx[i:i+t])), np.append(sty, np.mean(sy[i:i+t])), np.append(stz, np.mean(s[i:i+t]))
        xt = np.append(xt, i + t + start)

#     fig = plt.figure(2)
#     ax = fig.add_subplot(1, 1, 1)
#     major_ticks = np.arange(start + t, count+t, 200)
#     minor_ticks = np.arange(start + t, count+t, 50)
#     ax.set_xticks(major_ticks)
#     ax.set_xticks(minor_ticks, minor=True)
#     ax.grid(which='both')
#     plt.plot(xt, st, label='s')
#     plt.plot(xt, np.ones(st.shape[0])*np.mean(st[int(st.shape[0]/2.):]), label='mean(s)')
#     plt.plot(xt, stx, label='x')
#     plt.plot(xt, sty, label='y')
#     plt.plot(xt, stz, label='z')
#     plt.ylabel('s=mean(|$r_2 - r_1$|)')
#     plt.xlabel('t')
#     plt.legend()
#     plt.savefig('mean_s.png')
    return np.mean(st[int(st.shape[0]/2.):])

In [32]:
def peak(num_0,G):

    x0, y0, z0 = x[:,num_0], y[:,num_0], z[:,num_0]

    wx = np.cumsum(x0)
    wy = np.cumsum(y0)
    wz = np.cumsum(z0)
    totl = wx.shape[0]
    l_ = np.arange(25,175,25, dtype=int)
    
    peak_l = np.zeros(l_.shape[0])
#     header = ['t', 'peak']

    plt.clf()
    plt.subplots(figsize=(20, 10),facecolor='white')
#     plt.suptitle('Num=% s, r=%.2f R'%(num_0,np.sqrt(np.mean(x0**2+y0**2+z0**2)/R**2)))

    for i,l in enumerate(l_):

        peak_x = (wx[0:totl-2*l] - 2*wx[l:totl-l] + wx[2*l:totl])**2/l**2
        peak_y = (wy[0:totl-2*l] - 2*wy[l:totl-l] + wy[2*l:totl])**2/l**2
        peak_z = (wz[0:totl-2*l] - 2*wz[l:totl-l] + wz[2*l:totl])**2/l**2
        peak = peak_x + peak_y + peak_z
        totd = len(peak)
        time = np.arange(totd) + 2 * l
        peak_l[i] = np.mean(peak)
#         pd.DataFrame(np.array([time, peak]).transpose()).to_csv('Peak N={}, Г={}, l={}.csv'.format(x.shape[1], G, l), index=False, header=header)
        plt.subplot(2,3,i+1)
        plt.grid()
        sns.lineplot(time, peak).set(title = "l=%s" % (l))
        plt.plot(time, peak_l[i]*np.ones(peak.shape[0]), label="Mean=%.3f" % peak_l[i])
        plt.legend(fontsize=14)
        if i==0 or i==3:
            plt.ylabel('P(j)', fontsize=14)
        if i==1 or i==4:
            plt.xlabel('t', fontsize=14)
        
        
    plt.savefig('peak_{}_{} N={}'.format(num_0, G, x.shape[1])+string+'.png', bbox_inches='tight')

In [11]:
def kor_r():
    x0, y0, z0 = np.mean(x,axis=0), np.mean(y,axis=0), np.mean(z,axis=0)
    num_0 = np.argmin(x0**2 + y0**2 + z0**2)
    R = x.shape[1]**(1./3) * 1.5
    y_k = np.array([])
    x_k = np.array([])
    r_1 = np.sqrt((x[:,num_0]-np.mean(x[L:,num_0]))**2 + 
                  (y[:,num_0]-np.mean(y[L:,num_0]))**2 + 
                  (z[:,num_0]-np.mean(z[L:,num_0]))**2)
    
    for i in range(Np):
        if (i!=num_0):
            r = np.sqrt(np.mean((x[L:,num_0]-x[L:,i])**2+(y[L:,num_0]-y[L:,i])**2+(z[L:,num_0]-z[L:,i])**2))
            if (r <= R):
                x_k = np.append(x_k,r)
                r_2 = np.sqrt((x[:,i]-np.mean(x[L:,i]))**2 + 
                              (y[:,i]-np.mean(y[L:,i]))**2 + 
                              (z[:,i]-np.mean(z[L:,i]))**2)
                kor_ = np.sum((x[:,i]-np.mean(x[L:,i]))*(x[:,num_0]-np.mean(x[L:,num_0])) + 
                              (y[:,i]-np.mean(y[L:,i]))*(y[:,num_0]-np.mean(y[L:,num_0])) + 
                              (z[:,i]-np.mean(z[L:,i]))*(z[:,num_0]-np.mean(z[L:,num_0])))/np.sqrt(np.sum(r_1**2)*np.sum(r_2**2))
                y_k = np.append(y_k, kor_)
#     plt.figure(figsize=(5,5))
#     plt.scatter(x_k, y_k, alpha=1, s=20)
#     plt.xlabel('Расстояние между частицами r', fontsize=14)
#     plt.ylabel('k(r)', fontsize=14)
# #     plt.title('Корреляция двух частиц $k = \\frac{\sum r_{1i}r_{2i}}{\sqrt{\sum r_{1i}^2 \sum r_{2i}^2}}$')
#     plt.grid()
#     plt.savefig('cor_r_{}.png'.format(Np))
#     plt.show()
#     plt.close()
    
    return x_k,y_k

In [5]:
def kor_r_mean():
    
    x0, y0, z0 = np.mean(x,axis=0), np.mean(y,axis=0), np.mean(z,axis=0)
    num_0 = np.argmin(x0**2 + y0**2 + z0**2)
    R = x.shape[1]**(1./3) * 1.5
    y_k = np.zeros(int(x.shape[0])-1)
    x_k = np.zeros(int(x.shape[0])-1)
    
    st = 500
    for j in range(2000,int(x.shape[0])-st):
        s = 0
        r_1 = np.sqrt((x[j:j+st,num_0]-np.mean(x[j:j+st,num_0]))**2 + 
                  (y[j:j+st,num_0]-np.mean(y[j:j+st,num_0]))**2 + 
                  (z[j:j+st,num_0]-np.mean(z[j:j+st,num_0]))**2)
        for i in range(Np):
            if (i!=num_0):
                r = np.sqrt((x[j,i])**2+(y[j,i])**2+(z[j,i])**2)
                x_k[s] = x_k[s] + np.sqrt((x[j,num_0]-x[j,i])**2+(y[j,num_0]-y[j,i])**2+(z[j,num_0]-z[j,i])**2)
                r_2 = np.sqrt((x[j:j+st,i]-np.mean(x[j:j+st,i]))**2 + 
                              (y[j:j+st,i]-np.mean(y[j:j+st,i]))**2 + 
                              (z[j:j+st,i]-np.mean(z[j:j+st,i]))**2)
                kor_ = np.sum((x[j:j+st,i]-np.mean(x[j:j+st,i]))*(x[j:j+st,num_0]-np.mean(x[j:j+st,num_0])) + 
                                (y[j:j+st,i]-np.mean(y[j:j+st,i]))*(y[j:j+st,num_0]-np.mean(y[j:j+st,num_0])) + 
                              (z[j:j+st,i]-np.mean(z[j:j+st,i]))*(z[j:j+st,num_0]-np.mean(z[j:j+st,num_0])))/np.sqrt(np.sum(r_1**2)*np.sum(r_2**2))
                y_k[s] = y_k[s] + kor_ 
    #                 print(kor_)
                s += 1
    
    y_k = y_k[x_k!=0.] / (int(int(x.shape[0]) - st))
    x_k = x_k[x_k!=0.] / (int(int(x.shape[0]) - st))
#     print(y_k, x_k)
#     plt.figure(figsize=(5,5))
#     plt.scatter(x_k[x_k<=R], y_k[x_k<=R], alpha=1, s=20)
#     plt.xlabel('Расстояние между частицами r', fontsize=14)
#     plt.ylabel('k(r)', fontsize=14)
# #     plt.title('Корреляция двух частиц $k = \\frac{\sum r_{1i}r_{2i}}{\sqrt{\sum r_{1i}^2 \sum r_{2i}^2}}$')
#     plt.grid()
#     plt.savefig('cor_r_{}_{}_mean.png'.format(G, Np))
#     plt.show()
#     plt.close()
    return x_k[x_k<=R], y_k[x_k<=R]


In [6]:
def kor_v_mean():
    
    x0, y0, z0 = np.mean(x,axis=0), np.mean(y,axis=0), np.mean(z,axis=0)
    num_0 = np.argmin(x0**2 + y0**2 + z0**2)
    R = x.shape[1]**(1./3) * 1.5
    y_k = np.zeros(int(x.shape[0])-1)
    x_k = np.zeros(int(x.shape[0])-1)
    
    st = 500
    for j in range(2000,int(x.shape[0])-st):
        s = 0
        r_1 = np.sqrt((x[j:j+st,num_0]-np.mean(x[j:j+st,num_0]))**2 + 
                  (y[j:j+st,num_0]-np.mean(y[j:j+st,num_0]))**2 + 
                  (z[j:j+st,num_0]-np.mean(z[j:j+st,num_0]))**2)
        v_1 = np.sqrt((vx[j:j+st,num_0])**2 + (vy[j:j+st,num_0])**2 + (vz[j:j+st,num_0])**2)
        for i in range(Np):
            if (i!=num_0):
                x_k[s] = x_k[s] + np.sqrt((x[j,num_0]-x[j,i])**2+(y[j,num_0]-y[j,i])**2+(z[j,num_0]-z[j,i])**2)
                v_2 = np.sqrt((vx[j:j+st,i])**2 + (vy[j:j+st,i])**2 + (vz[j:j+st,i])**2)
                kor_ = np.sum((vx[j:j+st,i])*(vx[j:j+st,num_0]) + (vy[j:j+st,i])*(vy[j:j+st,num_0]) + (vz[j:j+st,i])*(vz[j:j+st,num_0]))/np.sqrt(np.sum(v_1**2)*np.sum(v_2**2))
                y_k[s] = y_k[s] + kor_ 
    #                 print(kor_)
                s += 1
    
    y_k = y_k[x_k!=0.] / (int(int(x.shape[0]) - st))
    x_k = x_k[x_k!=0.] / (int(int(x.shape[0]) - st))
#     print(y_k, x_k)
#     plt.figure(figsize=(5,5))
#     plt.scatter(x_k[x_k<=R], y_k[x_k<=R], alpha=1, s=20)
#     plt.xlabel('Расстояние между частицами r', fontsize=14)
#     plt.ylabel('k(v)', fontsize=14)
#     plt.grid()
#     plt.savefig('cor_v_{}_{}_mean.png'.format(G, Np))
    return x_k[x_k<=R], y_k[x_k<=R]


In [7]:
def kor_v():
    x0, y0, z0 = np.mean(x,axis=0), np.mean(y,axis=0), np.mean(z,axis=0)
    num_0 = np.argmin(x0**2 + y0**2 + z0**2)
    R = x.shape[1]**(1/3.) * 1.5
    rl = np.sqrt(x[L,:]**2 + y[L,:]**2 + z[L,:]**2)
    y_k = np.array([])
    x_k = np.array([])
    v_1 = np.sqrt((vx[:,num_0])**2 + (vy[:,num_0])**2 + (vz[:,num_0])**2)
    print(num_0)
    
    for i in range(Np):
        if (i!=num_0):
            r = np.sqrt((x[L,num_0]-x[L,i])**2+(y[L,num_0]-y[L,i])**2+(z[L,num_0]-z[L,i])**2)
            if r <= R:
                x_k = np.append(x_k,r)
                v_2 = np.sqrt((vx[:,i])**2 + (vy[:,i])**2 + (vz[:,i])**2)
                kor_ = np.sum((vx[:,i])*(vx[:,num_0]) + (vy[:,i])*(vy[:,num_0]) + (vz[:,i])*(vz[:,num_0]))/np.sqrt(np.sum(v_1**2)*np.sum(v_2**2))
                y_k = np.append(y_k, kor_)
    plt.figure(figsize=(5,5))
    plt.scatter(x_k, y_k, alpha=1, s=20)
    plt.xlabel('Расстояние между частицами r', fontsize=14)
    plt.ylabel('k(v)', fontsize=14)
#     plt.title('Корреляция двух частиц $k = \\frac{\sum v_{1i}v_{2i}}{\sqrt{\sum v_{1i}^2 \sum v_{2i}^2}}$')
    plt.grid()
    plt.savefig('cor_v_{}_{}.png'.format(G,Np))

In [8]:
def full(Np):
    R = (np.amax(x)+np.amax(y)+np.amax(z))/3.0
    for i in range(int(count/10)):
        a = x[i*10,:]
        b = y[i*10,:]
        c = z[i*10,:]

        r_d = ((max(a)+max(b)+max(c))/3.0)/(Np**(1/3.))
        
        xlist = np.linspace(-2*R/3.,2*R/3., 200)
        
        fig = plt.figure(2, figsize=(6, 6))
        plt.title('Шаг t={}'.format(i))
        plt.scatter(a[(np.abs(c)<r_d)], b[(np.abs(c)<r_d)], c='darkorange')
        plt.plot(xlist, np.sqrt((2*R/3)**2 - xlist**2), c='r')
        plt.plot(xlist, -np.sqrt((2*R/3)**2 - xlist**2), c='r')
        plt.Circle((0,0), 2*R/3., fill=False, )
        plt.xlabel('y')
        plt.ylabel('z')
        plt.legend
        plt.title("T = {}".format(i*10))
        plt.xlim(-(R+1), R+1)
        plt.ylim(-(R+1), R+1)
        plt.grid(linestyle='--')
        plt.savefig('yz_{}_{}.png'.format(i, num))
        plt.clf()
        
        out = cv2.VideoWriter(p+str(Np)+".avi", cv2.VideoWriter_fourcc(*"MJPG"), 20.0, (432,432)) #создаем видео
        for i in range(int(count/10)):
            out.write(cv2.imread(p+'yz_{}_1.png'.format(i)))
    
        out.release() #генерируем
        cv2.destroyAllWindows() #завершаем

In [9]:
def Radial(num_0,l, start,string):
    N = 200
    R = np.sqrt(np.amax((y)**2+(x)**2+(z)**2))*2
    h = R / N
    x_r = np.arange(N)*h + h/2.
    y_r = np.zeros((N, int((count-start )/l + 1)))
    print(y_r.shape)
    for k in range(start,count, l):
        s = int((k - start)/l)
        for i in range(Np):
            if (i!=num_0):
                j = int(np.sqrt((x[k,i]-x[k,num_0])**2 + (y[k,i]-y[k,num_0])**2 + (z[k,i]-z[k,num_0])**2) / h)
                y_r[j,s] = y_r[j,s] + 1
        
    y_rad = np.mean(y_r, axis=1)
    for j in range(N):
        y_rad[j] = y_rad[j]/(3 * h * (j * h)**2)
    
#     plt.clf()
#     plt.plot(x_r, y_rad)
#     plt.grid()
#     plt.savefig('rad_{}_{}_{}_'.format(start,l,num_0)+string+'.png')
#     plt.show()
#     plt.close()
    
#     open(p+'Rad.txt', 'a').close()
    return x_r, y_rad

In [33]:
G_date = np.arange(10,210, 10)
G_date = np.append(G_date, np.array([250, 500]))
# G_date = G_date.astype(int)

# G_date = np.arange(210,250, 10)
# fig, axs = plt.subplots(1,3,figsize=(15,5), sharex=True, sharey=True)
# t = ['a)', 'b)', 'c)']
N=1075
# data = np.array([])
G_date = [50, 100, 500]
    
for i, G in enumerate(G_date):
    print(G)

    count = 3000
    L = int(count*2/3.)
    # num = 1
#     G = 500
    p = '/Users/elizavetashpilko/Desktop/Work/test/HCP/N=1075/0/'+str(G)+'/'
#     plt.rcParams["savefig.directory"] = os.chdir(os.path.dirname('/Users/elizavetashpilko/Desktop/Work/test/HCP/'))

    x = np.loadtxt(p+"x_{}.txt".format(G), dtype='float64')
    y = np.loadtxt(p+"y_{}.txt".format(G), dtype='float64')
    z = np.loadtxt(p+"z_{}.txt".format(G), dtype='float64')
    vx = np.loadtxt(p+"v_x_{}.txt".format(G), dtype='float64')
    vy = np.loadtxt(p+"v_y_{}.txt".format(G), dtype='float64')
    vz = np.loadtxt(p+"v_z_{}.txt".format(G), dtype='float64')
    Np = x.shape[1]
    R = Np**(1./3)
    x0, y0, z0 = np.mean(x[L:,:],axis=0), np.mean(y[L:,:],axis=0), np.mean(z[L:,:],axis=0)
    num_0 = np.argmin(np.abs(np.sqrt(x0**2 + y0**2 + z0**2)))
    
    peak(num_0, G)
    
    
    
#     x_k, y_k = kor_v_mean()
#     data = np.array([x_k, y_k])
#     header = ['r', 'cor_v_mean']
#     pd.DataFrame(data.transpose()).to_csv('Cor_v_mean N={}, Г={}.csv'.format(Np, G),header=header, index=False)
    
#     x_k, y_k = kor_r_mean()
#     data = np.array([x_k, y_k])
#     header = ['r', 'cor_r_mean']
#     pd.DataFrame(data.transpose()).to_csv('Cor_r_mean N={}, Г={}.csv'.format(Np, G),header=header, index=False)
#     plt.subplot(1,3,i+1)
#     plt.scatter(x_k, y_k, alpha=1, s=20)
#     plt.title(t[i])
#     plt.grid()
# fig.text(0.5, 0.04, 'Расстояние между частицами r', ha='center', fontsize=14)
# fig.text(0.04, 0.5, 'k(r)', va='center', rotation='vertical', fontsize=14)
# plt.savefig('cor_r_{}_mean.png'.format(Np))

50
100
500


In [15]:
G = 500
N_date = np.array([150])
data = np.array([])
for N in N_date:
    L = -1
    p = '/Users/elizavetashpilko/Desktop/Work/test/HCP/N=150/'

    x = np.loadtxt(p+"x_{}.txt".format(G), dtype='float64')
    y = np.loadtxt(p+"y_{}.txt".format(G), dtype='float64')
    z = np.loadtxt(p+"z_{}.txt".format(G), dtype='float64')
    r = np.sqrt(x[L]**2 + y[L]**2 + z[L]**2)
    
    data = np.append(data, min(r))
    print(min(r))

0.1564938348464425


In [14]:
data

array([0.15649383])

In [37]:
N=335
p = '/Users/elizavetashpilko/Downloads/N='+str(N)+'/'

x = np.loadtxt(p+"x_{}.txt".format(G), dtype='float64')
y = np.loadtxt(p+"y_{}.txt".format(G), dtype='float64')
z = np.loadtxt(p+"z_{}.txt".format(G), dtype='float64')
x, y, z = x[L], y[L], z[L]

pd.DataFrame(np.array([x, y, z]).transpose()).to_csv('Claster.csv'.format(G), index=False, header=['x', 'y', 'z'])